In [52]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from math import*

#Custom functions
# Treatment_1 prend en argument le nom du fichier csv à traiter, 
# Supprime la colonne index si elle existe, supprime la colonne 'id'
# Renvoie un dataFrame de ce fichier (df)

def Treatment_1(name):
    global ID
    df = pd.read_csv(name, sep=";", header=None) #lire le fichier csv qui a un ";" comme délimiter,
    
 #Récuperer toutes les colonnes de la 1ère ligne du dataframe(df), les transformer en une liste (tolist()) puis les assigner 
    # aux colonnes du df
    df.columns = df.iloc[0, :].tolist()
    print(df.columns)
    
 #Supprimer la colonne index qui contient les indices de df ( de 0 au nombre max de colonnes) 
    #Pour cela on a appliqué la méthode drop avec la colonne 0 et axis 0: pour toutes les lignes 
    df = df.drop(0, axis = 0).reset_index(drop = True) # reset_index pour que les indices commencent par 0 
    ID = df.id #Récupérer la colonne id dans ID
    
 #Supprimer l'id pour chaque ligne: 
    df = df.drop('id', axis =  1)
    #retourner le dataset
    return df
#Fonction pour traitement de données, convertir en numeric, encoder les valeurs de type str, éliminer les unknown 
#et valeurs négatives
def Treatment_2(df):
    lb_make = LabelEncoder()      #Let's transform our string categorical data to numeric categorical data !    
    label_list = df.columns[5:7].tolist() # label list contient les colonnes 'prod_cost'(indice5), 'quality'(indice 6)
    wa = df.columns[7] #wa contient la colonne 'warranty'
    
    #numeric_list contient les colonnes d'indice different de [5,6,7,8]
    numeric_list = df.columns[~df.columns.isin(df.columns[5:8])].tolist()
    
        
    for x in df.columns:    #Parcourir la liste des colonnes avec la variable x,
        for w in df.index:  # parcourir la liste des lignes de chaque colonne avec la variable w
           # Si la ligne est de type type numerique: c'est a dire elle appartient au tableau numeric_list
            if x in numeric_list: 
            #Vérifier si une ligne de cette colonne est egale à 'unknown' et la remplacer par 0 (si oui)
                if df[x][w] == 'unknown':
                    df[x][w] = 0
            # si la colonne x = warranty: récupérer la uniquement le 1ercaractère: 
            #exemple si on a df[x][w] == '3_ans' ou '3ansss': garder uniquement 3 
            elif x in wa:
                df[x][w] = df[x][w][0]
            # si la colonne x est label_list( valeurs prod_cost et quality dans ce dataset) 
            elif x in label_list:
     #Ajouter des colonnes avec le nom composé de la concaténation de l'ancien nom de la colonne avec '_encodes' (ex: quality_encodes) et reçoit la valeur encodée en numéric
                df[str(x)+'_encoded'] = lb_make.fit_transform(df[x])
    # Si le dataset contient une colonne 'market_share': la supprimer
    if 'market_share' in df.columns.tolist():
        df = df.drop('market_share', axis = 1)
   #Supprimer les colonnes qui ont le nom dans label_list (quality et prod_cost qui ont des valeurs non numeriques) 
    df = df.drop(label_list, axis = 1)
    
    #Parcourir les colonnes du dataframe et les convertir en float     
    for x in df.columns:
        df[x] = df[x].astype(float)    
    return df # retourner le nouveau df qui contient que des valeurs numériques.

#Improve this treatment :)
#Dans cette fonction, nous allons remplacer les valeurs NaN, nulles ou négatives par la moyenne de la colonne:
#Prend en paramètres un dataframe et renvoie le meme df après traitement
def Treatment_3(df):
    # x prend la colonne prod_cost et y prend l'index de la colonne , parcourir toutes les valeurs  
    for x, y in zip(df.prod_cost, df.index): # zip permet de composer une structure avec plusieurs valeurs: dictionnaire, matrice ...
        # rep prend la valeur 1 si la valeur de prod_cost[y] est Nan(manquante)
        #Si la valeur est négative ou NaN ou 0: remplacer        
        df['prod_cost'][y] = df['prod_cost'][df.prod_cost.notnull()].astype('float64').mean()
    df['prod_cost']
    return df

#Files    
name_1 = 'mower_market_snapshot.csv'

In [53]:
Treatment_1(name_1)

Index(['capacity', 'failure_rate', 'id', 'margin', 'price', 'prod_cost',
       'product_type', 'quality', 'warranty', 'market_share',
       'attractiveness'],
      dtype='object')


,capacity,failure_rate,margin,price,prod_cost,product_type,quality,warranty,market_share,attractiveness
0,10.492623311991721,0.04252798088681092,633.7665842034021,828.8021034855201,195.03551928211797,auto-portee,Low,3ans,0.006360041749060895,0.6506482608150532
1,20.273730444538863,0.037896368225955296,731.5768165928985,882.894915941766,151.31809934886763,auto-portee,Low,3 ans.,0.006840423036959984,0.6997924743016689
2,20.19045691648912,0.039111245954194296,741.2182024972648,919.3555217086142,178.13731921134948,auto-portee,Low,3_ans,0.0061790166010061475,0.6321289330507346
3,27.220131037450034,0.04130311679867133,654.2556353742241,817.0850969501104,162.82946157588628,auto-portee,Low,3 anss,0.007924828620013936,0.8107298917700951
4,16.370911056135107,0.037938171867880455,703.0945265657465,844.5814645266526,141.48693796090603,auto-portee,Medium,3ans.,0.007405626037658374,0.7576141622593239
...,...,...,...,...,...,...,...,...,...,...
1394,44.959894548992835,0.2346469274422487,63.66256103796289,75.16110151903747,11.498540481074583,essence,Medium,1 an,0.0009725533691324659,0.5131263989569433
1395,59.35605810795307,0.253693682727865,64.0145430546866,75.39513179135517,11.380588736668567,essence,Low,1ans,0.001038155432543819,0.5477385360702411
1396,67.86522214290575,0.22893453099780575,62.132034909323984,810.883148411149,unknown,essence,Low,1an.,0.0012426320350569962,0.6556219140406752
1397,66.85543549976667,0.24494793928030933,58.697121576775956,68.49131835053741,9.794196773761458,essence,Low,1 an,0.001644284373573234,0.8675366784502783


In [54]:
df.columns[5:7]

Index(['product_type', 'quality'], dtype='object')

In [55]:
df.columns[7]

'warranty'

In [56]:
df.columns[~df.columns.isin(df.columns[5:8])]

Index(['capacity', 'failure_rate', 'margin', 'price', 'prod_cost',
       'market_share', 'attractiveness', 'product_type_encoded',
       'quality_encoded'],
      dtype='object')

In [57]:
zip(df.prod_cost, df.index)

In [58]:
Treatment_2(df)

<ipython-input-52-8421a32eb411>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x][w] = df[x][w][0]


,capacity,failure_rate,margin,price,prod_cost,warranty,attractiveness,product_type_encoded,quality_encoded
0,10.492623,0.042528,633.766584,828.802103,26.324028,3.0,0.650648,0.0,1.0
1,20.273730,0.037896,731.576817,882.894916,26.324026,3.0,0.699792,0.0,1.0
2,20.190457,0.039111,741.218202,919.355522,26.321771,3.0,0.632129,0.0,1.0
3,27.220131,0.041303,654.255635,817.085097,26.319527,3.0,0.810730,0.0,1.0
4,16.370911,0.037938,703.094527,844.581465,26.317295,3.0,0.757614,0.0,2.0
...,...,...,...,...,...,...,...,...,...
1394,44.959895,0.234647,63.662561,75.161102,26.191895,1.0,0.513126,2.0,2.0
1395,59.356058,0.253694,64.014543,75.395132,26.191791,1.0,0.547739,2.0,1.0
1396,67.865222,0.228935,62.132035,810.883148,26.191688,1.0,0.655622,2.0,1.0
1397,66.855435,0.244948,58.697122,68.491318,26.191587,1.0,0.867537,2.0,1.0


In [59]:
Treatment_3(df)

<ipython-input-52-8421a32eb411>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prod_cost'][y] = df['prod_cost'][df.prod_cost.notnull()].astype('float64').mean()


,capacity,failure_rate,margin,price,prod_cost,product_type,quality,warranty,market_share,attractiveness,product_type_encoded,quality_encoded
0,10.492623311991721,0.04252798088681092,633.7665842034021,828.8021034855201,26.1914,auto-portee,Low,3,0.006360041749060895,0.6506482608150532,0,1
1,20.273730444538863,0.037896368225955296,731.5768165928985,882.894915941766,26.1913,auto-portee,Low,3,0.006840423036959984,0.6997924743016689,0,1
2,20.19045691648912,0.039111245954194296,741.2182024972648,919.3555217086142,26.1912,auto-portee,Low,3,0.0061790166010061475,0.6321289330507346,0,1
3,27.220131037450034,0.04130311679867133,654.2556353742241,817.0850969501104,26.1911,auto-portee,Low,3,0.007924828620013936,0.8107298917700951,0,1
4,16.370911056135107,0.037938171867880455,703.0945265657465,844.5814645266526,26.191,auto-portee,Medium,3,0.007405626037658374,0.7576141622593239,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1394,44.959894548992835,0.2346469274422487,63.66256103796289,75.16110151903747,26.2136,essence,Medium,1,0.0009725533691324659,0.5131263989569433,2,2
1395,59.35605810795307,0.253693682727865,64.0145430546866,75.39513179135517,26.2137,essence,Low,1,0.001038155432543819,0.5477385360702411,2,1
1396,67.86522214290575,0.22893453099780575,62.132034909323984,810.883148411149,26.2137,essence,Low,1,0.0012426320350569962,0.6556219140406752,2,1
1397,66.85543549976667,0.24494793928030933,58.697121576775956,68.49131835053741,26.2137,essence,Low,1,0.001644284373573234,0.8675366784502783,2,1
